In [ ]:
import json
from datetime import datetime
from collections import Counter
import json
import pandas as pd
from scipy.stats import pearsonr
from datetime import datetime




file_path = '/content/DataEngineeringQ2.json'
with open(file_path, 'r') as file:
    data = json.load(file)



{'firstName': 0.0, 'lastName': 70.97, 'DOB': 32.26}


In [ ]:
data

[{'_id': 'T6hf3rb5',
  'appointmentId': '40d2-9c9f',
  'patientDetails': {'_id': 'T6hb630b3',
   'firstName': 'Css',
   'lastName': '',
   'emailId': '',
   'gender': '',
   'alternateContact': '',
   'birthDate': None},
  'phoneNumber': '96686896670',
  'consultationData': {'adviceTemplates': [],
   'advices': [],
   'attachments': [],
   'chiefComplaints': [],
   'customOne': [],
   'customThree': [],
   'customTwo': [],
   'disease': [],
   'doctorNotes': '',
   'emergencyInstructions': [],
   'emergencyInstructionsTemplate': [],
   'emrTemplates': [],
   'examinationNote': [],
   'findings': [],
   'investigationInstructions': [],
   'investigationTemplates': [],
   'investigations': [],
   'isBalicAppointment': False,
   'isQuickPrescription': False,
   'labTest': [],
   'languageCode': 'en',
   'medicineTemplates': [],
   'medicines': [{'medicineId': '619404',
     'medicineName': 'A',
     'medicineNameStrengthType': '',
     'frequency': '1-0-1',
     'duration': '90',
     'du

In [ ]:
patient_details = [entry['patientDetails'] for entry in data]

def calculate_missing_percentage(column_name):
    total = len(patient_details)
    missing = sum(1 for detail in patient_details if detail.get(column_name) in [None, "", " "])
    return (missing / total) * 100

missing_percentage = {
    'firstName': calculate_missing_percentage('firstName'),
    'lastName': calculate_missing_percentage('lastName'),
    'DOB': calculate_missing_percentage('birthDate')
}

missing_percentage = {k: round(v, 2) for k, v in missing_percentage.items()}


print(missing_percentage)


In [ ]:

from statistics import mode

genders = [detail.get('gender') for detail in patient_details]

gender_mode = mode([gender for gender in genders if gender not in [None, "", " "]])


imputed_genders = [gender if gender not in [None, "", " "] else gender_mode for gender in genders]


total = len(imputed_genders)
female_count = sum(1 for gender in imputed_genders if gender.lower() == 'female')
female_percentage = (female_count / total) * 100


female_percentage = round(female_percentage, 2)


print(female_percentage)


0.0


In [ ]:




with open(file_path, 'r') as file:
    data = json.load(file)

patient_details = [entry['patientDetails'] for entry in data]


def calculate_age(dob):
    if dob:
        birth_date = datetime.strptime(dob.split('T')[0], "%Y-%m-%d")
        today = datetime.today()
        return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    return None


def categorize_age(age):
    if age is None:
        return 'Unknown'
    if 0 <= age <= 12:
        return 'Child'
    elif 13 <= age <= 19:
        return 'Teen'
    elif 20 <= age <= 59:
        return 'Adult'
    elif age >= 60:
        return 'Senior'


for detail in patient_details:
    dob = detail.get('birthDate')
    age = calculate_age(dob)
    detail['ageGroup'] = categorize_age(age)


adult_count = sum(1 for detail in patient_details if detail['ageGroup'] == 'Adult')

print(f"Count of Adults: {adult_count}")


Count of Adults: 21


In [ ]:
patient_details = [entry['patientDetails'] for entry in data]

# Extract the number of medicines for each patient
def count_medicines(patient):
    return len(patient.get('medicines', []))

# Calculate the average number of medicines
total_medicines = sum(count_medicines(patient) for patient in patient_details)
average_medicines = total_medicines / len(patient_details) if patient_details else 0

print(f"Average number of medicines prescribed: {average_medicines:.2f}")


Average number of medicines prescribed: 0.00


In [ ]:


medicines = []
for record in data:
    consultation_data = record.get('consultationData', {})
    medicines_list = consultation_data.get('medicines', [])
    for medicine in medicines_list:
        medicines.append(medicine.get('medicineName', ''))


medicine_counter = Counter(medicines)


third_most_common_medicine = medicine_counter.most_common(3)[-1]

third_most_common_medicine



('C', 13)

In [ ]:

medicine_counts = [len(record.get('consultationData', {}).get('medicines', [])) for record in data]


average_medicines = sum(medicine_counts) / len(medicine_counts)

average_medicines


2.129032258064516

In [16]:

active_count = 0
inactive_count = 0

for record in data:
    medicines_list = record.get('consultationData', {}).get('medicines', [])
    for medicine in medicines_list:
        if medicine.get('isActive', False):
            active_count += 1
        else:
            inactive_count += 1


total_medicines = active_count + inactive_count


active_percentage = (active_count / total_medicines) * 100
inactive_percentage = (inactive_count / total_medicines) * 100

active_percentage, inactive_percentage


(69.6969696969697, 30.303030303030305)

In [ ]:
patient_info = []

for record in data:
    birth_date = record['patientDetails'].get('birthDate')
    if birth_date:
        birth_date = datetime.strptime(birth_date, '%Y-%m-%dT%H:%M:%S.%fZ')
        age = (datetime.now() - birth_date).days // 365
    else:
        age = None

    num_medicines = len(record['consultationData'].get('medicines', []))
    patient_info.append({'age': age, 'num_medicines': num_medicines})


df = pd.DataFrame(patient_info)


df = df.dropna(subset=['age'])


correlation, _ = pearsonr(df['age'], df['num_medicines'])


round(correlation, 2)

In [ ]:
patient_info = []

for record in data:
    birth_date = record['patientDetails'].get('birthDate')
    if birth_date:
        birth_date = datetime.strptime(birth_date, '%Y-%m-%dT%H:%M:%S.%fZ')
        age = (datetime.now() - birth_date).days // 365
    else:
        age = None

    num_medicines = len(record['consultationData'].get('medicines', []))
    patient_info.append({'age': age, 'num_medicines': num_medicines})


df = pd.DataFrame(patient_info)


df = df.dropna(subset=['age'])


correlation, _ = pearsonr(df['age'], df['num_medicines'])


round(correlation, 2)